# Statistical and Heuristic Approach (Pre 2000s)

This notebook shows detailed steps of Term Frequency-Inverse Document Frequency (TF-IDF) to identify most important words and sentences in a document. Sentences are ranked based on the sum of TF-IDF values of words in the sentence. Sentences with higher sum of TF-IDF values are considered more important and are extracted as summary of the document.

The notebook is organized as follows:
1. Load and Preprocess Data
2. Compute TF-IDF
3. Extract Summary

Note that, we’re implementing the actual algorithm here, not using any library to do the most of the tasks, we’re highly relying on the Math only.

The notebook is based on the following resources:
- [Text Summarization using TF-IDF](https://towardsdatascience.com/tf-idf-for-document-summary-7c7f91b3bb0c)

The notebook uses the following libraries:
- [NLTK](https://www.nltk.org/): Natural Language Toolkit
- [Pandas](https://pandas.pydata.org/): Data Analysis Library
- [Numpy](https://numpy.org/): Scientific Computing Library`

## Term Frequency
Term Frequency (TF) is a measure of how frequently a term occurs in a document. It is calculated as the number of times a term appears in a document divided by the total number of terms in the document.

$$TF(t) = \frac{Number\ of\ times\ term\ t\ appears\ in\ a\ document}{Total\ number\ of\ terms\ in\ the\ document}$$

## Inverse Document Frequency
Inverse Document Frequency (IDF) is a measure of how important a term is. It is calculated as the logarithm of the total number of documents divided by the number of documents containing the term.

$$IDF(t) = \log\left(\frac{Total\ number\ of\ documents}{Number\ of\ documents\ with\ term\ t\ in\ it}\right)$$

## TF-IDF
Term Frequency-Inverse Document Frequency (TF-IDF) is the product of TF and IDF. It is used to identify the importance of a term in a document relative to a collection of documents.

$$TF-IDF(t) = TF(t) \times IDF(t)$$

## Evaluation whether it is overfitting or not
- The algorithm is evaluated on the provided test data.
- The algorithm is evaluated based on the following metrics:
    - ROUGE-1
    - ROUGE-2
    - ROUGE-L

### Rouge-1
ROUGE-1 refers to the overlap of unigrams between the system and reference summaries.


### Rouge-2
ROUGE-2 refers to the overlap of bigrams between the system and reference summaries.

### Rouge-L
ROUGE-L is based on the longest common subsequence (LCS). One advantage of using LCS is that it does not require consecutive matches but in-sequence matches.

### Overfitting
The algorithm is considered overfitting if the ROUGE scores are significantly higher on the training data compared to the test data.

#### Example
Consider the following example:
Consider a document containing 100 words wherein the word 'cat' appears 3 times. The term frequency (TF) for 'cat' is then (3 / 100) = 0.03. 

Now, assume we have 10 million documents and the word 'cat' appears in one thousand of these. Then, the inverse document frequency (IDF) is calculated as log(10,000,000 / 1,000) = 4. Thus, the TF-IDF weight is the product of these quantities: 0.03 * 4 = 0.12.

##### Note
The notebook has comments at each step to make it easier to understand the implementation. Each step is explained in detail to make it easier to understand the implementation.

In [1]:
!pip3 install nltk
!pip3 install numpy
!pip3 install pandas

In [2]:
import math
import nltk
from nltk import word_tokenize,sent_tokenize, PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import numpy as np
from collections import defaultdict

In [3]:
# Download necessary NLTK data (only needed once)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anany\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
text="Modern economics does not differentiate between renewable and non-renewable materials, as its method is to measure everything by means of a money price. Thus, taking various alternate fuels, like coal, oil, wood or water power, the only difference between them recognised by modern economics is relative cost per equivalent unit. The cheapest is automatically the one to be preferred, as to do otherwise would be irrational and uneconomic. From a Buddhist point of view, of course, this will not do; the essential difference between non-renewable fuelslike coal and oil on the one hand and renewable fuels like wood and water-power on the other cannot be simply overlooked. Non-renewable goods must be used only if they are indispensable, and then only with the greatest care and the highest concern for conservation. To use them carelessly or extravagantly is an act of violence, and while complete non-violence may not be possible on this earth, it is nonetheless the duty of man to aim the ideal of non-violence in all he does."

In [5]:
def _get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts."""
    
    # Get the part of speech (POS) tag for the word
    tag = pos_tag([word])[0][1][0].upper()
    
    # Create a dictionary to map the POS tag to a format that WordNetLemmatizer accepts
    tag_dict = {"J": wordnet.ADJ,   # J is mapped to adjective
                "N": wordnet.NOUN,  # N is mapped to noun
                "V": wordnet.VERB,  # V is mapped to verb
                "R": wordnet.ADV,   # R is mapped to adverb
                "S": wordnet.ADJ_SAT # S is mapped to adjective satellite (similar to adjective)
                }
                
    # Return the mapped POS tag or default to NOUN if the tag isn't in the dictionary
    return tag_dict.get(tag, wordnet.NOUN)

In [6]:
def _create_frequency_matrix(sentences):
    """Create a frequency matrix for the given sentences."""
 
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        
        for word in words:
            word = word.lower()
            word = lemmatizer.lemmatize(word, _get_wordnet_pos(word))
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        # Use the full sentence as the key instead of truncating it
        frequency_matrix[sent] = freq_table

    return frequency_matrix


In [7]:
def _create_tf_matrix(freq_matrix):
    """Create a Term Frequency (TF) matrix from the frequency matrix."""
    tf_matrix = {}

    # Loop through each sentence's frequency table
    for sent, freq_table in freq_matrix.items():
        tf_table = {}

        # Get the total number of words in the sentence
        word_count_in_sentence = len(freq_table)

        # Loop through each word's frequency in the sentence
        for word, count in freq_table.items():
            # Calculate term frequency (TF) by dividing the word's count by the total number of words
            tf_table[word] = count / float(word_count_in_sentence)

        # Store the TF table in the TF matrix
        tf_matrix[sent] = tf_table
    print("TF Matrix:", tf_matrix)  # Debugging line
    return tf_matrix

In [8]:
def _create_idf_matrix(freq_matrix):
    """Create an Inverse Document Frequency (IDF) matrix from the frequency matrix."""
    idf_matrix = {}
    total_documents = len(freq_matrix)

    # Initialize an empty dictionary to count the number of documents containing each word
    word_document_counts = {}

    # Loop through each sentence's frequency table
    for freq_table in freq_matrix.values():
        for word in freq_table.keys():
            if word in word_document_counts:
                word_document_counts[word] += 1
            else:
                word_document_counts[word] = 1

    # Calculate IDF for each word
    for word, count in word_document_counts.items():
        idf_matrix[word] = math.log10(total_documents / float(count))
    print("IDF Matrix:", idf_matrix)  # Debugging line
    return idf_matrix


In [9]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    """Create a TF-IDF matrix from the TF and IDF matrices."""
    tf_idf_matrix = {}

    # Loop through each sentence's TF table
    for sent, tf_table in tf_matrix.items():
        tf_idf_table = {}

        # Loop through each word's TF value and multiply by its IDF value
        for word, tf_value in tf_table.items():
            tf_idf_table[word] = tf_value * idf_matrix.get(word, 0)

        # Store the TF-IDF table in the TF-IDF matrix
        tf_idf_matrix[sent] = tf_idf_table
    print("TF-IDF Matrix:", tf_idf_matrix)  # Debugging line
    return tf_idf_matrix

In [10]:
def _score_sentences(tf_idf_matrix):
    """Score each sentence based on its TF-IDF values."""
    sentence_scores = {}

    # Loop through each sentence's TF-IDF table
    for sent, tf_idf_table in tf_idf_matrix.items():
        # Sum the TF-IDF scores for each word in the sentence
        sentence_scores[sent] = sum(tf_idf_table.values())
    print("Sentence Scores:", sentence_scores)  # Debugging line
    return sentence_scores

In [11]:
def _generate_summary(sentences, sentence_scores, summary_length=3):
    """Generate a summary by selecting the top sentences based on their scores."""
    # Create a dictionary mapping sentence text to its index
    sentence_index_map = {sent: idx for idx, sent in enumerate(sentences)}
    
    # Sort sentences by their scores in descending order
    sorted_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)

    # Select the top N sentences for the summary
    top_sentences = sorted_sentences[:summary_length]

    # Ensure the sentences are correctly matched
    top_sentences_text = {sent[0] for sent in top_sentences}

    # Get the indices of the top sentences from the original list
    top_sentence_indices = [sentence_index_map[sent] for sent in sentences if sent in top_sentences_text]

    # Reorder the sentences based on their indices
    summary = [sentences[idx] for idx in sorted(top_sentence_indices)]

    return ' '.join(summary)


In [12]:
def summarize_text(text, summary_length=3):
    """Summarize the input text by selecting the top N sentences based on TF-IDF scores."""
    # Split text into sentences
    sentences = sent_tokenize(text)
    
    # Create frequency matrix
    freq_matrix = _create_frequency_matrix(sentences)
    
    # Calculate TF matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    
    # Calculate IDF matrix
    idf_matrix = _create_idf_matrix(freq_matrix)
    
    # Calculate TF-IDF matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    
    # Score sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    
    # Generate summary
    summary = _generate_summary(sentences, sentence_scores, summary_length)
    
    return summary


In [14]:
summary = summarize_text(text, summary_length=3)
print("Summary:")
print(summary)

TF Matrix: {'Modern economics does not differentiate between renewable and non-renewable materials, as its method is to measure everything by means of a money price.': {'modern': 0.07142857142857142, 'economics': 0.07142857142857142, 'differentiate': 0.07142857142857142, 'renewable': 0.07142857142857142, 'non-renewable': 0.07142857142857142, 'material': 0.07142857142857142, ',': 0.07142857142857142, 'method': 0.07142857142857142, 'measure': 0.07142857142857142, 'everything': 0.07142857142857142, 'mean': 0.07142857142857142, 'money': 0.07142857142857142, 'price': 0.07142857142857142, '.': 0.07142857142857142}, 'Thus, taking various alternate fuels, like coal, oil, wood or water power, the only difference between them recognised by modern economics is relative cost per equivalent unit.': {'thus': 0.045454545454545456, ',': 0.22727272727272727, 'take': 0.045454545454545456, 'various': 0.045454545454545456, 'alternate': 0.045454545454545456, 'fuel': 0.045454545454545456, 'like': 0.04545454